In [12]:
# Imports

import pandas as pd
import calendar
import plotly.express as px
import hvplot.pandas
from sqlalchemy import create_engine
import sqlalchemy as sal
import matplotlib.pyplot as plt

In [13]:
# Connection to Postgre database

engine = create_engine("postgresql://{0}:{1}@{2}/{3}".format("postgres","dynasty","localhost:5432","Credit_Card_Fraud_Detection"))

In [14]:
# Data fetching of transactions by CH2 and CH18 

query = "SELECT * FROM cardholder_transactions"
transactions_by_cardholder_df = pd.read_sql(sql=query, con=engine)
transactions_by_cardholder_df['amount']= transactions_by_cardholder_df['amount'].str.replace("$", "",regex=True)
transactions_by_cardholder_df['amount']= transactions_by_cardholder_df['amount'].str.replace(",", "",regex=True)
transactions_by_cardholder_df['amount'] = transactions_by_cardholder_df['amount'].astype(float)
cardholders_2_and_18_transactions= (transactions_by_cardholder_df.loc[transactions_by_cardholder_df.ch_id.isin([2, 18])]
                                    .drop(columns=['t_id', 'name'])
                                    )
cardholders_2_and_18_transactions.head()


,ch_id,card,date,amount
133,2,4866761290278198714,2018-06-24,4.96
134,2,4866761290278198714,2018-10-30,5.87
135,2,4866761290278198714,2018-05-01,6.55
136,2,4866761290278198714,2018-09-25,3.92
137,2,4866761290278198714,2018-03-06,4.10


In [15]:
# Analysis of transactions by CH2 using hvplot - Output in 'Outputs' dir

cardholder_2_transactions = (
    cardholders_2_and_18_transactions[cardholders_2_and_18_transactions['ch_id'] == 2]
    .drop(columns='ch_id')
)

# Date vs. Amount on sum
ch2_transactions_by_date = cardholder_2_transactions[['date', 'amount']]
ch2_total_transactions_by_date = ch2_transactions_by_date.groupby('date').sum()

# Plot for CH2
ch2_total_transactions_by_date.hvplot(title="Sum of Transaction Amounts for Card Holders 2", rot=45)

:Curve   [date]   (amount)

In [16]:
# Analysis of transactions by CH18 using hvplot - Output in 'Outputs' dir

cardholder_18_transactions = (
    cardholders_2_and_18_transactions[cardholders_2_and_18_transactions['ch_id'] == 18]
    .drop(columns='ch_id')
)

# Date vs. Amount on sum
ch18_transactions_by_date = cardholder_18_transactions[['date', 'amount']]
ch18_total_transactions_by_date = ch18_transactions_by_date.groupby('date').sum()

# Plot for CH18
ch18_total_transactions_by_date.hvplot(title="Sum of Transaction Amounts for Card Holders 18", rot=45)


:Curve   [date]   (amount)

In [17]:
# Combined analysis of transactions by CH2 and CH18 using hvplot - Output in 'Outputs' dir

ch2_total_transactions_by_date.hvplot(xlabel='Date',
                       ylabel='Amount',
                       label="Card Holder 2") * ch18_total_transactions_by_date.hvplot(xlabel='Date',
                                                                        ylabel='Amount',
                                                                        rot=45,
                                                                        label="Card Holder 18")

:Overlay
   .Curve.Card_Holder_2  :Curve   [date]   (amount)
   .Curve.Card_Holder_18 :Curve   [date]   (amount)

In [18]:
# Fetching daily transactions from JAN'18 to JUNE'18 for CH25

# Slicing the data
cardholder_25_transactions = (transactions_by_cardholder_df.loc[transactions_by_cardholder_df.ch_id.isin([25])]
                                    .drop(columns=['t_id', 'name', 'card'])
                                    )

# Changing date column type to datetime and slicing data from JAN to JUNE
cardholder_25_transactions['date'] = pd.to_datetime(cardholder_25_transactions['date'])
cardholder_25_transactions.set_index('date', inplace=True)
cardholder_25_transactions.sort_index(ascending=True, inplace=True)
ch25_transactions_jan_jun = cardholder_25_transactions.loc['2018-01-01':'2018-07-01'].reset_index()


ch25_transactions_jan_jun_grp = (ch25_transactions_jan_jun.groupby('date').sum()
                                 .drop(columns='ch_id')
                                )
ch25_transactions_jan_jun_grp.head()

,amount
date,
2018-01-02,1.46
2018-01-05,10.74
2018-01-07,2.93
2018-01-10,1.39
2018-01-14,17.84


In [19]:
# Numeric months to month names
from datetime import datetime
ch25_transactions_jan_jun['date'] = (pd.to_datetime(ch25_transactions_jan_jun['date'], format='%Y%m%d')
                                     .dt.strftime('%b')
                                    )
ch25_box_plot_df = ch25_transactions_jan_jun.drop(columns='ch_id')
ch25_box_plot_df.head()

,date,amount
0,Jan,1.46
1,Jan,10.74
2,Jan,2.93
3,Jan,1.39
4,Jan,17.84


In [20]:
# Generating the six box plots using plotly express
fig = px.box(ch25_box_plot_df, x='date',
             y='amount',
            title="Transactions By Month for Card Holder 25")
fig.show()